In [21]:
import torch

def clean_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

# Usage example
clean_gpu_memory()

In [2]:
import transformers
from transformers import pipeline
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)


messages = [
    {"role": "system", "content": "You are a translator who translate LaTeX into spoken English."},
    {"role": "user", "content": r"Translate the LaTeX into spoken English: x = \frac{{-b \pm \sqrt{{b^2 - 4ac}}}}{2a}"},
]

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipe(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)


Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.64s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\bitsandbytes\nn\modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


The LaTeX code "\frac{{-b \pm \sqrt{{b^2 - 4ac}}}}{2a}" translates to:

"X equals negative b, plus or minus the square root of b-squared minus 4a-c, all divided by 2a."


In [2]:
import transformers
from transformers import pipeline
import torch
import concurrent.futures

# 모델 초기화 함수
def initialize_pipeline(model_id):
    return transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={
            "torch_dtype": torch.float16,
            "quantization_config": {"load_in_4bit": True},
            "low_cpu_mem_usage": True,
        },
    )

# 모델 ID
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# 메시지 및 종료 토큰 설정
messages = [
    {"role": "system", "content": "You are a translator who translate LaTeX into spoken English."},
    {"role": "user", "content": r"Translate the LaTeX into spoken English: x = \frac{{-b \pm \sqrt{{b^2 - 4ac}}}}{2a}"},
]

def translate_with_pipeline(pipe, messages):
    terminators = [
        pipe.tokenizer.eos_token_id,
    ]
    outputs = pipe(
        messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"]

# 파이프라인 초기화
num_pipelines = 2  # 병렬로 사용할 모델 파이프라인의 수
pipelines = [initialize_pipeline(model_id) for _ in range(num_pipelines)]

def find_assistant_message(messages):
    for message in messages:
        if message['role'] == 'assistant':
            return message['content']
    return None

def extract_only_data(data):
    if '"' in data:
        parts = data.split('"')
        result = parts[-2] if len(parts) >= 3 else data
    elif '\n' in data:
        lines = data.split('\n')
        result = lines[-1] if lines else data
        
    elif 'sorry' in data or 'cannot' in data or 'apologize' in data or '$' in data:
        result = "None"
    else:
        result = data
    return result

# 병렬 처리
with concurrent.futures.ThreadPoolExecutor(max_workers=num_pipelines) as executor:
    future_to_pipe = {executor.submit(translate_with_pipeline, pipe, messages): pipe for pipe in pipelines}
    for future in concurrent.futures.as_completed(future_to_pipe):
        pipe = future_to_pipe[future]
        try:
            result = future.result()
            result = find_assistant_message(result)
            result = extract_only_data(result)
            print(result)
        except Exception as exc:
            print(f'{pipe} generated an exception: {exc}')


<>:59: SyntaxWarning: invalid escape sequence '\ '
<>:59: SyntaxWarning: invalid escape sequence '\ '
C:\Users\wjdrb\AppData\Local\Temp\ipykernel_53464\3147869073.py:59: SyntaxWarning: invalid escape sequence '\ '
  """
Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.89s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:33<00:00,  8.37s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


X equals negative B, plus or minus, the square root of B squared minus 4AC, all divided by 2A.
x equals negative b, plus or minus the square root of b-squared minus 4ac, all divided by 2a


In [1]:
import pandas as pd
df = pd.read_parquet('df_not_len_5_cleaned_unique_eq.parquet', engine='fastparquet')
print("Load complete")

Load complete


In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import concurrent.futures
import transformers
import torch
import time

# 모델 초기화 함수
def initialize_pipeline(model_id):
    return transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={
            "torch_dtype": torch.float16,
            "quantization_config": {"load_in_4bit": True},
            "low_cpu_mem_usage": True,
        },
    )

# 번역 함수
def translate_with_pipeline(pipe, text):
    terminators = [
        pipe.tokenizer.eos_token_id,
    ]
    messages = [
        {"role": "system", "content": "You are a translator who translate LaTeX into spoken English."},
        {"role": "user", "content": f"Translate the LaTeX into spoken English: {text}"},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"]

# 어시스턴트 메시지 추출 함수
def find_assistant_message(messages):
    for message in messages:
        if message['role'] == 'assistant':
            return message['content']
    return None

# 데이터 추출 함수
def extract_only_data(data):
    if '"' in data:
        parts = data.split('"')
        result = parts[-2] if len(parts) >= 3 else data
    elif '\n' in data:
        lines = data.split('\n')
        result = lines[-1] if lines else data
    elif 'sorry' in data or 'cannot' in data or 'apologize' in data or '$' in data:
        result = "None"
    else:
        result = data
    return result

# 병렬로 번역하고 결과를 저장하는 함수
def process_and_save(df, pipe, file_idx, times):
    results = []
    for eq in df['equation']:
        try:
            start_time = time.time()
            result = translate_with_pipeline(pipe, eq)
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)
            result = find_assistant_message(result)
            result = extract_only_data(result)
            results.append(result)
        except Exception as e:
            results.append(None)
            print(f"Error processing equation {eq}: {e}")

    df['translated'] = results
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f'translated_part_{file_idx}.parquet')

# 데이터프레임을 파이프라인 개수로 나누는 함수
def split_dataframe(df, n):
    return [df[i::n] for i in range(n)]

# 모델 ID 및 파이프라인 초기화
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
num_pipelines = 2
pipelines = [initialize_pipeline(model_id) for _ in range(num_pipelines)]



c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.69s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.55s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:

# 데이터프레임 로드
df = pd.read_parquet('df_not_len_5_cleaned_unique_eq.parquet', engine='fastparquet')

# 데이터프레임 나누기
dfs = split_dataframe(df, num_pipelines)

# 평균 시간을 저장할 리스트
times = []

# 병렬 처리 및 저장
with concurrent.futures.ThreadPoolExecutor(max_workers=num_pipelines) as executor:
    futures = []
    for i, (pipe, df_part) in enumerate(zip(pipelines, dfs)):
        futures.append(executor.submit(process_and_save, df_part, pipe, i, times))

    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as exc:
            print(f'Generated an exception: {exc}')

# 예상 완료 시간 계산
if times:
    avg_time_per_eq = sum(times) / len(times)
    total_eq = len(df)
    total_time = avg_time_per_eq * total_eq
    print(f"Estimated completion time: {total_time / 60:.2f} minutes")
else:
    print("No timings collected, cannot estimate completion time.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\bitsandbytes\nn\modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-e

In [2]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import transformers
import torch
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio

# 이벤트 루프 중첩을 허용
nest_asyncio.apply()

# 모델 초기화 함수
def initialize_pipeline(model_id):
    return transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={
            "torch_dtype": torch.float16,
            "quantization_config": {"load_in_4bit": True},
            "low_cpu_mem_usage": True,
        },
    )

# 번역 함수
def translate_with_pipeline(pipe, text):
    terminators = [
        pipe.tokenizer.eos_token_id,
    ]
    messages = [
        {"role": "system", "content": "You are a translator who translate LaTeX into spoken English."},
        {"role": "user", "content": f"Translate the LaTeX into spoken English: {text}"},
    ]
    outputs = pipe(
        messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"]

# 어시스턴트 메시지 추출 함수
def find_assistant_message(messages):
    for message in messages:
        if message['role'] == 'assistant':
            return message['content']
    return None

# 데이터 추출 함수
def extract_only_data(data):
    if '"' in data:
        parts = data.split('"')
        result = parts[-2] if len(parts) >= 3 else data
    elif '\n' in data:
        lines = data.split('\n')
        result = lines[-1] if lines else data
    elif 'sorry' in data or 'cannot' in data or 'apologize' in data or '$' in data:
        result = "None"
    else:
        result = data
    return result

# 병렬로 번역하고 결과를 저장하는 함수
async def process_and_save(df, pipe, file_idx, times, executor):
    loop = asyncio.get_event_loop()
    results = []

    for eq in df['equation']:
        start_time = time.time()
        result = await loop.run_in_executor(executor, translate_with_pipeline, pipe, eq)
        elapsed_time = time.time() - start_time
        times.append(elapsed_time)

        result = find_assistant_message(result)
        result = extract_only_data(result)
        results.append(result)
        print(result)  # 생성된 텍스트 출력

    df['translated'] = results
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f'translated_part_{file_idx}.parquet')

# 데이터프레임을 파이프라인 개수로 나누는 함수
def split_dataframe(df, n):
    return [df[i::n] for i in range(n)]

# 메인 함수
async def main():
    model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
    num_pipelines = 2
    pipelines = [initialize_pipeline(model_id) for _ in range(num_pipelines)]

    df = pd.read_parquet('df_not_len_5_cleaned_unique_eq.parquet', engine='fastparquet')
    dfs = split_dataframe(df, num_pipelines)

    times = []

    with ThreadPoolExecutor(max_workers=num_pipelines) as executor:
        tasks = [
            process_and_save(df_part, pipe, i, times, executor)
            for i, (pipe, df_part) in enumerate(zip(pipelines, dfs))
        ]
        await asyncio.gather(*tasks)

    if times:
        avg_time_per_eq = sum(times) / len(times)
        total_eq = len(df)
        total_time = avg_time_per_eq * total_eq
        print(f"Estimated completion time: {total_time / 60:.2f} minutes")
    else:
        print("No timings collected, cannot estimate completion time.")

# 실행
if __name__ == "__main__":
    asyncio.run(main())


Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.50s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.12s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\bitsandbytes\nn\modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transforme

deta


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


is in the interval 3, 7


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

A function of t.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


X naught is equal to X of t.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


delta t
